In [1]:
import pandas as pd

In [2]:
data_url = '../../data/code-summarization/'

train_data_url = data_url + 'train.jsonl'
valid_data_url = data_url + 'valid.jsonl'
test_data_url = data_url + 'test.jsonl'

In [3]:
train_data = pd.read_json(path_or_buf=train_data_url, lines=True)
valid_data = pd.read_json(path_or_buf=valid_data_url, lines=True)
test_data = pd.read_json(path_or_buf=test_data_url, lines=True)

In [4]:
train_data.head(5)

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,Harium/keel,src/main/java/com/harium/keel/effect/height/So...,Sobel.apply,@Override\n public ImageSource apply(ImageS...,java,@Override\n public ImageSource apply(ImageS...,"[@, Override, public, ImageSource, apply, (, I...",Expects a height mat as input\n\n@param input ...,"[Expects, a, height, mat, as, input]",0369ae674f9e664bccc5f9e161ae7e7a3b949a1e,https://github.com/Harium/keel/blob/0369ae674f...,train
1,skuzzle/jeve,jeve/src/main/java/de/skuzzle/jeve/providers/E...,EventStackImpl.popEvent,public <L extends Listener> void popEvent(Even...,java,public <L extends Listener> void popEvent(Even...,"[public, <, L, extends, Listener, >, void, pop...",Pops the top event off the current event stack...,"[Pops, the, top, event, off, the, current, eve...",42cc18947c9c8596c34410336e4e375e9fcd7c47,https://github.com/skuzzle/jeve/blob/42cc18947...,train
2,skuzzle/jeve,jeve/src/main/java/de/skuzzle/jeve/stores/Abst...,AbstractSynchronizedListenerSource.modify,protected void modify(Transaction t) {\n ...,java,protected void modify(Transaction t) {\n ...,"[protected, void, modify, (, Transaction, t, )...",Executes the given transaction within the cont...,"[Executes, the, given, transaction, within, th...",42cc18947c9c8596c34410336e4e375e9fcd7c47,https://github.com/skuzzle/jeve/blob/42cc18947...,train
3,skuzzle/jeve,jeve/src/main/java/de/skuzzle/jeve/stores/Abst...,AbstractSynchronizedListenerSource.read,protected <E> E read(Supplier<E> sup) {\n ...,java,protected <E> E read(Supplier<E> sup) {\n ...,"[protected, <, E, >, E, read, (, Supplier, <, ...",Executes the given supplier within the context...,"[Executes, the, given, supplier, within, the, ...",42cc18947c9c8596c34410336e4e375e9fcd7c47,https://github.com/skuzzle/jeve/blob/42cc18947...,train
4,vnesek/nmote-iim4j,src/main/java/com/nmote/iim4j/stream/SubIIMInp...,SubIIMInputStream.setOffsetAndLength,"protected void setOffsetAndLength(long offset,...",java,"protected void setOffsetAndLength(long offset,...","[protected, void, setOffsetAndLength, (, long,...",This should be called from a subclass construc...,"[This, should, be, called, from, a, subclass, ...",ec55b02fc644cd722e93051ac0bdb96b00cb42a8,https://github.com/vnesek/nmote-iim4j/blob/ec5...,train


In [6]:
type(train_data['code_tokens'][0])

list

In [7]:
train_syntax_error_ids = []
valid_syntax_error_ids = []
test_syntax_error_ids = []

In [8]:
from tqdm import tqdm
import javalang
from javalang.ast import Node

In [9]:
def parse_program(func):
    tokens = javalang.tokenizer.tokenize(func)
    parser = javalang.parser.Parser(tokens)
    tree = parser.parse_member_declaration()
    return tree

In [10]:
for i in tqdm(range(len(train_data['code']))):
    try:
        tree = parse_program(train_data['code'][i])
    except:
        train_syntax_error_ids.append(i)


100%|██████████| 164923/164923 [04:37<00:00, 594.30it/s]


In [11]:
len(train_syntax_error_ids)

109

In [12]:
valid_syntax_error_ids = []
for i in tqdm(range(len(valid_data['code']))):
    try:
        tree = parse_program(valid_data['code'][i])
    except:
        valid_syntax_error_ids.append(i)

100%|██████████| 5183/5183 [00:07<00:00, 720.49it/s]


In [13]:
len(valid_syntax_error_ids)

4

In [14]:
for i in tqdm(range(len(test_data['code']))):
    try:
        tree = parse_program(test_data['code'][i])
    except:
        test_syntax_error_ids.append(i)

100%|██████████| 10955/10955 [00:18<00:00, 601.78it/s]


In [15]:
len(test_syntax_error_ids)

3

In [16]:
train_data_new = train_data.drop(train_syntax_error_ids)
valid_data_new = valid_data.drop(valid_syntax_error_ids)
test_data_new = test_data.drop(test_syntax_error_ids)

In [18]:
new_data_url = '../../data/code-summarization-new/'

train_data_new.to_json(path_or_buf=new_data_url+'train.jsonl', orient='records', lines=True)
valid_data_new.to_json(path_or_buf=new_data_url+'valid.jsonl',  orient='records', lines=True)
test_data_new.to_json(path_or_buf=new_data_url+'test.jsonl',  orient='records', lines=True)


In [25]:
print('train', len(train_data_new), 'valid', len(valid_data_new), 'test', len(test_data_new))

train 164814 valid 5179 test 10952


In [3]:
small_data_url = '../../data/code-summarization-small/'
middle_data_url = '../../data/code-summarization-middle/'

In [2]:
new_data_url = '../../data/code-summarization-new/'
train_data_new = pd.read_json(new_data_url+'train.jsonl', lines=True)
valid_data_new = pd.read_json(new_data_url+'valid.jsonl', lines=True)
test_data_new = pd.read_json(new_data_url+'test.jsonl', lines=True)

train_data_small = train_data_new[0:1000]
valid_data_small = valid_data_new[0:100]
test_data_small = test_data_new[0:100]

train_data_middle = train_data_new.sample(frac=0.1)
valid_data_middle = valid_data_new.sample(frac=0.1)
test_data_middle = test_data_new.sample(frac=0.1)

In [21]:
train_data_new.head(5)

,repo,path,func_name,original_string,language,code,code_tokens,docstring,docstring_tokens,sha,url,partition
0,Harium/keel,src/main/java/com/harium/keel/effect/height/So...,Sobel.apply,@Override\n public ImageSource apply(ImageS...,java,@Override\n public ImageSource apply(ImageS...,"[@, Override, public, ImageSource, apply, (, I...",Expects a height mat as input\n\n@param input ...,"[Expects, a, height, mat, as, input]",0369ae674f9e664bccc5f9e161ae7e7a3b949a1e,https://github.com/Harium/keel/blob/0369ae674f...,train
1,skuzzle/jeve,jeve/src/main/java/de/skuzzle/jeve/providers/E...,EventStackImpl.popEvent,public <L extends Listener> void popEvent(Even...,java,public <L extends Listener> void popEvent(Even...,"[public, <, L, extends, Listener, >, void, pop...",Pops the top event off the current event stack...,"[Pops, the, top, event, off, the, current, eve...",42cc18947c9c8596c34410336e4e375e9fcd7c47,https://github.com/skuzzle/jeve/blob/42cc18947...,train
2,skuzzle/jeve,jeve/src/main/java/de/skuzzle/jeve/stores/Abst...,AbstractSynchronizedListenerSource.modify,protected void modify(Transaction t) {\n ...,java,protected void modify(Transaction t) {\n ...,"[protected, void, modify, (, Transaction, t, )...",Executes the given transaction within the cont...,"[Executes, the, given, transaction, within, th...",42cc18947c9c8596c34410336e4e375e9fcd7c47,https://github.com/skuzzle/jeve/blob/42cc18947...,train
3,skuzzle/jeve,jeve/src/main/java/de/skuzzle/jeve/stores/Abst...,AbstractSynchronizedListenerSource.read,protected <E> E read(Supplier<E> sup) {\n ...,java,protected <E> E read(Supplier<E> sup) {\n ...,"[protected, <, E, >, E, read, (, Supplier, <, ...",Executes the given supplier within the context...,"[Executes, the, given, supplier, within, the, ...",42cc18947c9c8596c34410336e4e375e9fcd7c47,https://github.com/skuzzle/jeve/blob/42cc18947...,train
4,vnesek/nmote-iim4j,src/main/java/com/nmote/iim4j/stream/SubIIMInp...,SubIIMInputStream.setOffsetAndLength,"protected void setOffsetAndLength(long offset,...",java,"protected void setOffsetAndLength(long offset,...","[protected, void, setOffsetAndLength, (, long,...",This should be called from a subclass construc...,"[This, should, be, called, from, a, subclass, ...",ec55b02fc644cd722e93051ac0bdb96b00cb42a8,https://github.com/vnesek/nmote-iim4j/blob/ec5...,train


In [23]:
type(train_data_new['docstring_tokens'][0])

list

In [24]:
train_data_small.to_json(path_or_buf=small_data_url+'train.jsonl', orient='records', lines=True)
valid_data_small.to_json(path_or_buf=small_data_url+'valid.jsonl',  orient='records', lines=True)
test_data_small.to_json(path_or_buf=small_data_url+'test.jsonl',  orient='records', lines=True)

In [4]:
train_data_middle.to_json(path_or_buf=middle_data_url+'train.jsonl', orient='records', lines=True)
valid_data_middle.to_json(path_or_buf=middle_data_url+'valid.jsonl',  orient='records', lines=True)
test_data_middle.to_json(path_or_buf=middle_data_url+'test.jsonl',  orient='records', lines=True)

Encoding with 'utf-8'

In [2]:
import pandas as pd

In [3]:
data_url = ['../../data/code-summarization-enhanced-full/', '../../data/code-summarization-enhanced-middle/', '../../data/code-summarization-enhanced-small/']


In [11]:
def del_non_utf8(data):
    for i in range(len(data)):
        code = data['code'][i]
        data['code'][i] = data['code'][i].encode('utf-8','ignore').decode("utf-8")
        if code != data['code'][i]:
            print('i')

In [12]:
for i in range(3):
    train_data_url = data_url[i] + 'train.jsonl'
    valid_data_url = data_url[i] + 'valid.jsonl'
    test_data_url = data_url[i] + 'test.jsonl'

    train_data = pd.read_json(train_data_url, lines=True)
    valid_data = pd.read_json(valid_data_url, lines=True)
    test_data = pd.read_json(test_data_url, lines=True)

    del_non_utf8(train_data)
    del_non_utf8(test_data)
    del_non_utf8(valid_data)

    with open(data_url[i]+'train_utf8.jsonl', 'w', encoding='utf-8') as file:
        train_data.to_json(file, force_ascii=False, orient='records', lines=True)
    
    with open(data_url[i]+'valid_utf8.jsonl', 'w', encoding='utf-8') as file:
        valid_data.to_json(file, force_ascii=False, orient='records', lines=True)

    with open(data_url[i]+'test_utf8.jsonl', 'w', encoding='utf-8') as file:
        test_data.to_json(file, force_ascii=False, orient='records', lines=True)

In [13]:
'\ud800asdadfadfs'.encode('utf-8','ignore').decode("utf-8")

''

In [14]:
'\ud800' == ''

False

In [15]:
'asdadfadfs'.encode('utf-8','ignore').decode("utf-8")

'asdadfadfs'

In [16]:
'\ud800asdadfadfs'.encode('utf-8','ignore').decode("utf-8")

'asdadfadfs'

get the max node num

In [1]:
import pandas as pd 
enhance_data_url = '../../data/code-summarization-enhanced-full/'
train_data_new = pd.read_json(enhance_data_url+'train_utf8.jsonl', lines=True)

In [4]:
train_data_new['code'][81588]

'@Override\n    public int propertyNameToCType(byte[]bytes, int p, int end) {\n        Integer ctype = PosixBracket.PBSTableUpper.get(bytes, p, end);\n        if (ctype != null) return ctype;\n        throw new CharacterPropertyException(EncodingError.ERR_INVALID_CHAR_PROPERTY_NAME, bytes, p, end - p);\n    }'